<a href="https://colab.research.google.com/github/mmistroni/TensorFlowPlayground/blob/master/HousePrices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Authenticate User </h3>

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
# importing libraries
import pandas as pd


<h3>FEtching data from clo9ud storage </h3>

In [23]:
!gsutil cp gs://mm_tensorflow_bucket/pp-2018-part2.csv	 /tmp/pp-2018-part2.csv
!gsutil cp gs://mm_tensorflow_bucket/pp-2015-part1.csv	 /tmp/pp-2015-part1.csv
  
  

Copying gs://mm_tensorflow_bucket/pp-2018-part2.csv...
- [1 files][ 84.7 MiB/ 84.7 MiB]                                                
Operation completed over 1 objects/84.7 MiB.                                     
Copying gs://mm_tensorflow_bucket/pp-2015-part1.csv...
- [1 files][ 83.4 MiB/ 83.4 MiB]                                                
Operation completed over 1 objects/83.4 MiB.                                     


<h3> Readign the dataset from Land Registry</h3>

In [0]:
def read_land_registry_datasets(filename):
  
  df =  pd.read_csv(filename, names=['unique_id',	'price_paid','deed_date','postcode',
                                                       'property_type',	'new_build',	'estate_type',	
                                                       'saon',	'paon',	'street','locality',
                                                       'town',	'district',	'county',	'transaction_category','linked_data_uri']).fillna('') 
  print ('Found {} in {}'.format(df.shape[0], filename))
  return df

In [0]:
from datetime import datetime
def find_sales_for_postcode(postcode, filenames):
  print('Finding sold prices for postcode:{}'.format(postcode))
  lr_datasets = [read_land_registry_datasets(filename) for filename in filenames]
  
  results = []
  for df in lr_datasets:
    restricted = df[df['postcode'].str.contains('BN2 5')][['price_paid', 'deed_date', 'postcode', 'property_type', 'saon', 'paon', 'street']]
    flats_only = restricted[restricted['property_type'] == 'F']
    print('found {} flats for postcode {} '.format(flats_only.shape[0], postcode))
    results.append(flats_only)
    
  all_res =  pd.concat(results)
  print('All res has shape{}'.format(all_res.shape))
  return all_res

                                        

In [36]:
filenames = ['/tmp/pp-2018-part2.csv', '/tmp/pp-2015-part1.csv']
final = find_sales_for_postcode('BN2 5', filenames)
print('Exportign to csv...with {}'.format(final.shape))
filename = '/tmp/Results.csv'.format(datetime.now().strftime('%Y%m%d-%H:%M'))
final.to_csv(filename, header=True)


Finding sold prices for postcode:BN2 5
Found 511575 in /tmp/pp-2018-part2.csv
Found 504743 in /tmp/pp-2015-part1.csv
found 32 flats for postcode BN2 5 
found 50 flats for postcode BN2 5 
All res has shape(82, 7)
Exportign to csv...with (82, 7)


In [37]:

print('Copying to bucket')
!gsutil cp /tmp/Results.csv gs://mm_tensorflow_bucket/                                       


Copying to bucket
Copying file:///tmp/Results.csv [Content-Type=text/csv]...
/ [1 files][  5.7 KiB/  5.7 KiB]                                                
Operation completed over 1 objects/5.7 KiB.                                      


In [0]:
# Get gsc service
def get_gcs_service():
  from googleapiclient.discovery import build
  gcs_service = build('storage', 'v1')
  return gcs_service

In [44]:
def download_from_storage(bucketname, bucketfilename, tmpfile):
  from apiclient.http import MediaIoBaseDownload

  gcs_service = get_gcs_service()
  with open(tmpfile, 'wb') as f:
    request = gcs_service.objects().get_media(bucket=bucketname,
                                              object=bucketfilename)
    media = MediaIoBaseDownload(f, request)

    done = False
    while not done:
      # _ is a placeholder for a progress object that we ignore.
      # (Our file is small, so we skip reporting progress.)
      _, done = media.next_chunk()

  print('Download complete')
  
filename = "/tmp/frombucket.csv"
download_from_storage("mm_tensorflow_bucket", "Results.csv",   filename)
  
  
  

Download complete


In [45]:
!cat /tmp/frombucket.csv

,price_paid,deed_date,postcode,property_type,saon,paon,street
6080,135000,2018-09-28 00:00,BN2 5JR,F,36B,,BRISTOL GARDENS
8072,500000,2018-05-04 00:00,BN2 5RA,F,"OCEAN HEIGHTS, 40",FLAT 6,ROEDEAN ROAD
12033,191500,2018-10-10 00:00,BN2 5UX,F,STARBOARD COURT,FLAT 22,
16020,518000,2018-09-26 00:00,BN2 5SH,F,SOVEREIGN COURT,FLAT 39,THE STRAND
16029,352500,2018-09-20 00:00,BN2 5XB,F,70,,VICTORY MEWS
18019,365000,2018-08-30 00:00,BN2 5YW,F,"ST AUGUSTINES, 14",FLAT 8,THE LEES
58744,512734,2018-06-12 00:00,BN2 5ZF,F,"SIRIUS, 6",APARTMENT 82,THE BOARDWALK
98217,325000,2018-03-02 00:00,BN2 5XB,F,81,,VICTORY MEWS
110882,350000,2018-01-05 00:00,BN2 5TQ,F,F16,,MARINE GATE
123934,118000,2018-04-16 00:00,BN2 5UB,F,COURCELS,25,ARUNDEL STREET
221213,337500,2018-08-28 00:00,BN2 5SH,F,SOVEREIGN COURT,FLAT 15,THE STRAND
225176,500000,2018-08-15 00:00,BN2 5WE,F,WELLINGTON COURT,FLAT 8,THE STRAND
231149,562500,2018-08-10 00:00,BN2 5SH,F,SOVEREIGN COURT,FLAT 38,THE STRAND
241587,125000,2018-11-08 00:00,BN2 5